In [92]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import docx
from docx.shared import Inches
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

#Set working directory to desired outpur folder
os.chdir('O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files')

#list of files to work on. These must be generated from the other pyhton programs I made
files = ['O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 23°C.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 23C Conditioned.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 -40°C.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 80°C.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 110°C.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 125°C.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 150°C.xlsx',
         'O:\\Tech_Service\\Analytical - New\\Analytical\\TAR Data\\2019\\19-0075 Spec. development GMMPD for Akulon K-FKG3 BK00001- Deans 3-19-2019\\python files\\19-0075 170°C.xlsx',]
names = ['23C', '23C Conditioned', '-40C', '80C', '110C', '125C', '150C', '170C']


#pulls modulus, stress, strain values from the reports above and creates plot of all "sample 1's"
moduli = []
moduli_dev = []
stress = []
stress_dev = []
strain = []
strain_dev = []
plt.figure(figsize = (10,5))
for i in range(len(list(files))):
    df = pd.read_excel(files[i], sheet_name = 'Report')
    mod = df.loc['Modulus (MPa)', 'AVG']
    mod_dev = df.loc['Modulus (MPa)', 'DEV'].round(1)
    moduli.append(mod)
    moduli_dev.append(mod_dev)
    stress_ab = df.loc['Stress at Break (MPa)', 'AVG'].round(1)
    stress_ab_dev = df.loc['Stress at Break (MPa)', 'DEV'].round(1)
    stress.append(stress_ab)
    stress_dev.append(stress_ab_dev)
    strain_ab = df.loc['Strain at Break (%)', 'AVG'].round(2)
    strain_ab_dev = df.loc['Strain at Break (%)', 'DEV'].round(1)
    strain.append(strain_ab)
    strain_dev.append(strain_ab_dev)
    
    max_stress = max(stress)
    stress_axis_limit = max_stress*1.1
    max_strain = max(strain)
    strain_axis_limit = max_strain*1.1
    
    plotdf = pd.read_excel(files[i], sheet_name = 'Sample1RawData')
    plotstrain = plotdf['(%)']
    plotstress = plotdf['(MPa)']
    grd = pd.Series(np.gradient(plotstress))
    grd = grd.where(grd>-1).dropna()

    plt.plot(plotstrain[0:len(grd)], plotstress[0:len(grd)])
    plt.legend(labels = names)
    plt.xlabel('Tensile Strain (%)', fontsize=16)
    plt.ylabel('Tensile Stress (MPa)', fontsize=16)
    plt.ylim(ymin=0, ymax = stress_axis_limit)
    plt.xlim(xmin=0, xmax = strain_axis_limit)
    plt.title('19-0107', fontsize=20)
    plt.savefig('Overlay')
plt.close()


properties = pd.Series(['Modulus (MPa)', 'Stdev', 'Stress at Break (MPa)', 'Stdev', 'Strain at Break (%)', 'Stdev']).T

#creates a dataframe of moduli & break/yield properties to write to TAR report-style docx table
alldata = [stress, stress_dev, strain, strain_dev, moduli, moduli_dev]
alldata = pd.DataFrame(alldata, index = ['Modulus (MPa)', 'Stdev', 'Stress at Break (MPa)', 'Stdev', 'Strain at Break (%)', 'Stdev'],  columns = names)
with pd.ExcelWriter('test.xlsx') as writer:
    alldata.to_excel(writer, sheet_name = 'Report')
    writer.sheets['Report'].insert_image('A10', 'Overlay.png')

#creates word file that will contain table
doc = docx.Document()
font = doc.styles['Normal'].font
font.name = 'Arial'

t = doc.add_table(alldata.shape[0]+2, alldata.shape[1]+2)

for i in range(alldata.shape[-1]):
    t.cell(1,i+2).text = alldata.columns[i]
for i in range(alldata.shape[0]):
    for j in range (alldata.shape[-1]):
        t.cell(i+2, j+2).text = str(alldata.values[i,j])
        
#helper functions to underline rows and columns
def make_rows_underline(*rows):
    for row in rows:
        for cell in row.cells:
             for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.underline = True
def make_columns_underline(*columns):
    for column in columns:
            for cell in column.cells:
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        run.font.underline = True
                        
#merging cells and writing lables
t.cell(2,1).merge(t.cell(3,1))
t.cell(4,1).merge(t.cell(5,1))
t.cell(6,1).merge(t.cell(7,1))
t.cell(0,1).merge(t.cell(0,9))
t.cell(0,0).merge(t.cell(1,0))
t.cell(3,1).text = 'ITSB'
t.cell(5,1).text = 'ITEBB'
t.cell(7,1).text = 'ITMXG'
t.cell(2,0).text = 'Stress at Break (MPa)'
t.cell(3,0).text = 'Stdev'
t.cell(4,0).text = 'Strain at Break (MPa)'
t.cell(5,0).text = 'Stdev'
t.cell(6,0).text = 'Modulus (MPa)'
t.cell(7,0).text = 'Stdev'
t.cell(0,1).text = 'Akulon® K-FKG3 BK00001'
t.cell(1,1).text = 'Test Code'

#underlining
make_rows_underline(t.rows[1])
make_columns_underline(t.columns[0])

#centers text
for row in t.rows:
    for cell in row.cells:
        paragraphs = cell.paragraphs
        for paragraph in paragraphs:
            for run in paragraph.runs:
                font = run.font
                font.size= Pt(10.5)
            paragraph_format = paragraph.paragraph_format     
            paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
#insert DSM logo
pic='logo.png'
paragraph = t.cell(0,0).paragraphs[0]
run = paragraph.add_run()
run.add_picture(pic, height=Inches(.50), width=Inches(1.47))

#bold and enlarge table title (grade name)
title_p = t.cell(0,1).paragraphs[0].runs[0]
font = title_p.font
font.size = Pt(18)
font.bold = True

t.style = 'Table Grid'
t.font = 'Arial'
doc.save('test.docx')
    

In [93]:
import os
os.path

<module 'ntpath' from 'C:\\ProgramData\\Anaconda3\\lib\\ntpath.py'>